In [2]:
!pip install tflearn tensorflow sentence-transformers gradio spacy googletrans==4.0.0-rc1 nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py

In [3]:
import spacy
import googletrans
from googletrans import Translator

nlp = spacy.load("en_core_web_sm")
translator = Translator()

In [4]:
import gradio as gr
import json
import random
import torch
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from googletrans import Translator

nlp = spacy.load("en_core_web_sm")
translator = Translator()

model_biobert = "dmis-lab/biobert-base-cased-v1.1"
model = AutoModelForQuestionAnswering.from_pretrained(model_biobert)
tokenizer = AutoTokenizer.from_pretrained(model_biobert)

with open('new_intents8.json', 'r') as file:
    intents = json.load(file)

sim_model = SentenceTransformer('all-MiniLM-L6-v2')

for intent in intents['intents']:
    intent['pattern_embeddings'] = [sim_model.encode([pattern])[0] for pattern in intent['patterns']]

def get_response(query):
    query_embed = sim_model.encode([query])[0]
    max_similar = -1
    chosen_intent = None

    for intent in intents['intents']:
        for pattern_embed in intent['pattern_embeddings']:
            similar = cosine_similarity([query_embed], [pattern_embed])[0][0]

            if similar > max_similar:
                max_similar = similar
                chosen_intent = intent

    if chosen_intent is not None:
        return chosen_intent['responses'][0]

    return "I'm sorry, I don't have an answer for that."

def user(query, history):
    response = get_response(query)

    doc = nlp(query)
    query_corrected = " ".join([token.lemma_ for token in doc if token.lemma_ != "-PRON-" and token.is_alpha and token.shape_ != "X"])

    detected_language = translator.detect(query_corrected).lang
    print(detected_language)
    if detected_language != "en":
        response_translated = translator.translate(response, dest="en").text
    else:
        response_translated = response

    doc_corrected = nlp(response_translated)
    final_response = " ".join([token.lemma_ for token in doc_corrected if token.lemma_ != "-PRON-" and token.is_alpha and token.shape_ != "X"])

    return final_response, history + [[query, final_response]]

with gr.Blocks(title="FirstAidGenie", head="FirstAidGenie", theme="dark", css=".container { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; }") as demo:
    gr.Markdown('''<h1>First Aid Genie</h1>''')
    chatbot = gr.Chatbot()

    msg = gr.Textbox(label="How are you feeling:", placeholder="How are you...")
    submit = gr.Button("Send")
    clear = gr.Button("Clear")


    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)
    submit.click(user, inputs=[msg, chatbot], outputs=[msg, chatbot])
demo.queue()
demo.launch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:530: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-660c4b63-738b105b37d6143f49584c3d;16dee55e-3782-437f-8fb7-74db49827afa)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8df580ab244b2924e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [19]:
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
import nltk
nltk.download('punkt')
reference = "To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row."

reference_tokenize = word_tokenize(reference)

generated = get_response("I am not feeling well from last few days. It could be due to heat. I think I have fever.")
generated_tokenize = word_tokenize(generated)
bleu_score = sentence_bleu([reference_tokenize], generated_tokenize)

print("BLEU score:", bleu_score)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


BLEU score: 1.0
